#### expected test MIND data format
{
"past_articles": [
{'article_id': UUID,
"title": "title1", 
"content": "content",
"url": "url 1" 
}
],

"todays_articles": [
{'article_id': UUID,
"title": "title1", 
"url": "url 1" 
}, 
], 

"click_data": {
UUID: [list of articel ids]
}

"num_recs": int

}

In [4]:
import pandas as pd
from tqdm import tqdm
from os import path
import random
import numpy as np
import csv
from ast import literal_eval
import json
from pydantic import BaseModel
from uuid import UUID, uuid4
from datetime import datetime, timezone
from pydantic import BaseModel, UUID4
from typing import List


In [9]:
# 加载original news和behavior 数据
val_behavior_df = pd.read_table("/home/XLL1713/POPROX/engine0/NRMS_bert/data/mind/large/val/behaviors.tsv",
               header='infer',
               usecols=range(5),
               names=[
                   'impression_id', 'user', 'time',
                   'clicked_news', 'impressions'
               ])

val_behavior_df.fillna('', inplace = True)

val_news_df = pd.read_table("/home/XLL1713/POPROX/engine0/NRMS_bert/data/mind/large/val/news.tsv", 
            header = 'infer', 
            usecols = range(4),
            names = ['id', 'topic', 'subtopic', 'title'])

print(f"The total unique uuid is {val_behavior_df.shape[0] + val_news_df.shape[0]}")

The total unique uuid is 448494


In [30]:
uuids = [uuid4() for _ in range(448494)]
print(f"There are {len(set(uuids))} unique uuids are generated")

There are 448494 unique uuids are generated


In [32]:
# 将uuid assign到news id和impression id中
val_news_df['uuid'] = uuids[:val_news_df.shape[0]]
val_news_df['str_uuid'] = val_news_df['uuid'].apply(lambda x: str(x))

val_behavior_df['uuid'] = uuids[val_news_df.shape[0]:]
val_behavior_df['str_uuid'] = val_behavior_df['uuid'].apply(lambda x: str(x))


In [33]:
val_behavior_df.head(1)

,impression_id,user,time,clicked_news,impressions,uuid,str_uuid
0,1,U134050,11/15/2019 8:55:22 AM,N12246 N128820 N119226 N4065 N67770 N33446 N10...,N91737-0 N30206-0 N54368-0 N117802-0 N18190-0 ...,cb6630a1-0150-4ef1-b944-de32af2ac598,cb6630a1-0150-4ef1-b944-de32af2ac598


In [42]:
# 保存一些数据
ID_title = dict(zip(val_news_df.id, val_news_df.title))
ID_newsuuid = dict(zip(val_news_df.id, val_news_df.uuid))

news_struuid_ID = dict(zip(val_news_df.str_uuid, val_news_df.id)) # uuid - news id
behavior_struuid_ID = dict(zip(val_behavior_df.str_uuid, val_behavior_df.impression_id)) # uuid - impression id

with open('./news_uuid_ID.json', 'w') as file:
    json.dump(news_struuid_ID , file, indent=4)
with open('./behavior_uuid_ID.json', 'w') as file:
    json.dump(behavior_struuid_ID , file, indent=4)

In [37]:
class Article(BaseModel):
    article_id: UUID
    title: str
    content: str = ""
    url: str
    published_at: datetime = datetime(1970, 1, 1, 0, 0, tzinfo=timezone.utc)

class ClickHistory(BaseModel):
    account_id: UUID4 = None
    article_id: List[UUID4]
        
def convert_to_Article(data: dict) -> Article:
    if 'published_at' not in data:
        data['published_at'] = datetime(1970, 1, 1, 0, 0, tzinfo=timezone.utc)
    return Article(**data)

def convert_to_ClickHistory(data):
    return ClickHistory(**data)


#print('---',article.model_dump())
#print('---',Article.parse_obj(article.model_dump()))


In [38]:
test_list = []

In [54]:
# todays_articles (candidates)
# ID_newsuuid 

for i in tqdm(range(val_behavior_df.shape[0])): 
    test_json = {}
    test_json['past_articles'] = [] 
    test_json['num_recs'] = 10 
    test_json['todays_articles'] = [] # list of json 


    row = val_behavior_df.iloc[i]
    
    impression_uuid = row.str_uuid

    for candidate_pair in row.impressions.split(' '): 
        single_news = {}
        single_news['article_id'] = ID_newsuuid[candidate_pair.split('-')[0]]
        single_news['url'] = str(ID_newsuuid[candidate_pair.split('-')[0]])
        single_news['title'] = single_news['content'] = ID_title[candidate_pair.split('-')[0]]
        
        single_news = convert_to_Article(single_news)
        single_news = single_news.model_dump()
        test_json['todays_articles'].append(single_news)
        
    
    for article in row.clicked_news.split(): 
        single_news = {}
        single_news['article_id'] =  ID_newsuuid[article]
        single_news['url'] = str(ID_newsuuid[article])
        single_news['title'] = single_news['content'] = ID_title[article]
        
        single_news = convert_to_Article(single_news)
        single_news = single_news.model_dump()
        test_json['past_articles'].append(single_news)
    
    click_data = {'account_id': impression_uuid, 'article_id': [ID_newsuuid[id] for id in row.clicked_news.split()]}
 
    click_data = convert_to_ClickHistory(click_data) 
    
    click_data = click_data.model_dump()
    test_json['click_data'] = click_data
    

    test_list.append(test_json)
    
    

100%|█████████████████████████████████| 376471/376471 [03:42<00:00, 1691.49it/s]


In [55]:
len(test_list)

376471

In [56]:
import json
def custom_encoder(obj):
    if isinstance(obj, UUID):
        return str(obj)
    elif isinstance(obj, datetime):
        return obj.isoformat()

with open('./mind_test.json', 'w') as file:
    json.dump(test_list, file, default=custom_encoder, indent=4)


In [57]:
with open('./mind_test.json', 'r') as json_file:
    data_loaded = json.load(json_file)

print(len(data_loaded))


376471


In [ ]:
# val_behavior_df[val_behavior_df['impressions'] == 'N29160-0 N25621-0 N122640-1 N46894-0 N54368-0 N18190-0 N55801-0']